In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# read the file exported from Deliverable 1 
merged_df = pd.read_csv('output_file.csv')

In [3]:
counts_agent_tone = merged_df['agent_tone'].value_counts()
print(counts_agent_tone)

neutral       39688
calm          22882
frustrated     3438
angry           366
polite           81
Name: agent_tone, dtype: int64


In [4]:
merged_df

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript,agent_tone,customer_tone,average_sentiment,silence_percent_average,AHT,AST,primary_call_reason,call_hour,elite_level_code,AST_Category,AHT_Category
0,4667960400,2033123310,963118,2024-07-31 23:56:00,2024-08-01 00:03:00,2024-08-01 00:34:00,\n\nAgent: Thank you for calling United Airlin...,neutral,angry,-0.04,0.39,1860.0,420.0,voluntary cancel,23,4.0,Normal,Long
1,1122072124,8186702651,519057,2024-08-01 00:03:00,2024-08-01 00:06:00,2024-08-01 00:18:00,\n\nAgent: Thank you for calling United Airlin...,calm,neutral,0.02,0.35,720.0,180.0,booking,0,0.0,Normal,Long
2,6834291559,2416856629,158319,2024-07-31 23:59:00,2024-08-01 00:07:00,2024-08-01 00:26:00,\n\nAgent: Thank you for calling United Airlin...,neutral,polite,-0.13,0.32,1140.0,480.0,irrops,23,0.0,Long,Long
3,2266439882,1154544516,488324,2024-08-01 00:05:00,2024-08-01 00:10:00,2024-08-01 00:17:00,\n\nAgent: Thank you for calling United Airlin...,neutral,frustrated,-0.20,0.20,420.0,300.0,upgrade,0,2.0,Normal,Normal
4,1211603231,5214456437,721730,2024-08-01 00:04:00,2024-08-01 00:14:00,2024-08-01 00:23:00,\n\nAgent: Thank you for calling United Airlin...,neutral,polite,-0.05,0.35,540.0,600.0,seating,0,0.0,Long,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66450,7569738090,7367304988,783441,2024-08-31 23:51:00,2024-08-31 23:57:00,2024-09-01 00:07:00,\n\nAgent: Thank you for calling United Airlin...,neutral,frustrated,-0.19,0.51,600.0,360.0,mileage plus,23,2.0,Normal,Normal
66451,1563273072,8022667294,413148,2024-08-31 23:48:00,2024-08-31 23:57:00,2024-09-01 00:25:00,\n\nAgent: Thank you for calling United Airlin...,calm,calm,0.05,0.34,1680.0,540.0,post flight,23,0.0,Long,Long
66452,8865997781,4474867021,980156,2024-08-31 23:55:00,2024-08-31 23:58:00,2024-09-01 00:06:00,\n\nAgent: Thank you for calling United Airlin...,calm,frustrated,0.03,0.22,480.0,180.0,upgrade,23,1.0,Normal,Normal
66453,8019240181,9762042472,616988,2024-08-31 23:52:00,2024-08-31 23:58:00,2024-09-01 00:04:00,\n\nAgent: Thank you for calling United Airlin...,calm,polite,0.05,0.42,360.0,360.0,upgrade,23,0.0,Normal,Normal


In [5]:
counts_customer_tone = merged_df['customer_tone'].value_counts()
print(counts_customer_tone)

neutral       13366
polite        13351
angry         13313
calm          13236
frustrated    13189
Name: customer_tone, dtype: int64


In [6]:
# primary reasons for calls
reason_counts = merged_df['primary_call_reason'].value_counts().reset_index()
print(reason_counts)


                    index  primary_call_reason
0                  irrops                13264
1        voluntary change                10827
2                 seating                 6355
3            mileage plus                 5837
4             post flight                 4309
5          communications                 3836
6   products and services                 2848
7                 baggage                 2808
8                 upgrade                 2734
9                 booking                 2624
10               check in                 1899
11               checkout                 1882
12       voluntary cancel                 1603
13        digital support                 1220
14                    etc                  950
15       traveler updates                  932
16           other topics                  816
17        schedule change                  730
18    products & services                  475
19             disability                  402
20    unaccom

In [7]:
# Approach for optimization in operations
# First of all we are going to categories all the reasons into 5 categories, Category_1, Category_2, Category_3, Category_4, category_5
# We will assign heavy work to category_1 and least heavy work to category_5 respectively
# By heavy work we mean reasons which have highest number of calls with respect to reason
# Now we will rate agent on the basis of their AHT and tone, 
# The polite agents are the best agents and the angry are the worst agents but in both extreme cases the AHT is short so we are not considering AHT to judge an agent
# Now there are 2 factor that defines the work is heavy:
# 1. Customer tone, 2. Long AHT (As customers are approximately equally distributed according to their tone, we can negect it) 
# There are 1 factors which can be used to assign Team to manage optimum category
# 1. Agent Tone
# By all this we mean that category_1 will have most number of calls and reasons which have highest AHT,
# Category_1 will be assigned to a team in which there are more number of people and efficient agents who have good ratings
# Same goes for Category_2, 3, and 4 in descending order 
# After that we will make a reason classifier

In [8]:
# looks like irrops, voluntary change, seating, mileage plus, post flight, communications are the most important reasons for calls.
# ranking them in decending order
reason_counts.columns = ['primary_call_reason', 'count']
reason_counts['rank'] = reason_counts['count'].rank(ascending=False, method='dense')
merged_df = merged_df.merge(reason_counts[['primary_call_reason', 'rank']], on='primary_call_reason', how='left')
merged_df

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript,agent_tone,customer_tone,average_sentiment,silence_percent_average,AHT,AST,primary_call_reason,call_hour,elite_level_code,AST_Category,AHT_Category,rank
0,4667960400,2033123310,963118,2024-07-31 23:56:00,2024-08-01 00:03:00,2024-08-01 00:34:00,\n\nAgent: Thank you for calling United Airlin...,neutral,angry,-0.04,0.39,1860.0,420.0,voluntary cancel,23,4.0,Normal,Long,13.0
1,1122072124,8186702651,519057,2024-08-01 00:03:00,2024-08-01 00:06:00,2024-08-01 00:18:00,\n\nAgent: Thank you for calling United Airlin...,calm,neutral,0.02,0.35,720.0,180.0,booking,0,0.0,Normal,Long,10.0
2,6834291559,2416856629,158319,2024-07-31 23:59:00,2024-08-01 00:07:00,2024-08-01 00:26:00,\n\nAgent: Thank you for calling United Airlin...,neutral,polite,-0.13,0.32,1140.0,480.0,irrops,23,0.0,Long,Long,1.0
3,2266439882,1154544516,488324,2024-08-01 00:05:00,2024-08-01 00:10:00,2024-08-01 00:17:00,\n\nAgent: Thank you for calling United Airlin...,neutral,frustrated,-0.20,0.20,420.0,300.0,upgrade,0,2.0,Normal,Normal,9.0
4,1211603231,5214456437,721730,2024-08-01 00:04:00,2024-08-01 00:14:00,2024-08-01 00:23:00,\n\nAgent: Thank you for calling United Airlin...,neutral,polite,-0.05,0.35,540.0,600.0,seating,0,0.0,Long,Normal,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66450,7569738090,7367304988,783441,2024-08-31 23:51:00,2024-08-31 23:57:00,2024-09-01 00:07:00,\n\nAgent: Thank you for calling United Airlin...,neutral,frustrated,-0.19,0.51,600.0,360.0,mileage plus,23,2.0,Normal,Normal,4.0
66451,1563273072,8022667294,413148,2024-08-31 23:48:00,2024-08-31 23:57:00,2024-09-01 00:25:00,\n\nAgent: Thank you for calling United Airlin...,calm,calm,0.05,0.34,1680.0,540.0,post flight,23,0.0,Long,Long,5.0
66452,8865997781,4474867021,980156,2024-08-31 23:55:00,2024-08-31 23:58:00,2024-09-01 00:06:00,\n\nAgent: Thank you for calling United Airlin...,calm,frustrated,0.03,0.22,480.0,180.0,upgrade,23,1.0,Normal,Normal,9.0
66453,8019240181,9762042472,616988,2024-08-31 23:52:00,2024-08-31 23:58:00,2024-09-01 00:04:00,\n\nAgent: Thank you for calling United Airlin...,calm,polite,0.05,0.42,360.0,360.0,upgrade,23,0.0,Normal,Normal,9.0


In [9]:
# Lets categorize it in a way that category 1 gets the most work and category 5 the least 
# 1 - 5 loop and reapeating it till all entries
merged_df['category_by_reason'] = ((merged_df['rank'] - 1) % 5) + 1

In [10]:
merged_df

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript,agent_tone,customer_tone,average_sentiment,silence_percent_average,AHT,AST,primary_call_reason,call_hour,elite_level_code,AST_Category,AHT_Category,rank,category_by_reason
0,4667960400,2033123310,963118,2024-07-31 23:56:00,2024-08-01 00:03:00,2024-08-01 00:34:00,\n\nAgent: Thank you for calling United Airlin...,neutral,angry,-0.04,0.39,1860.0,420.0,voluntary cancel,23,4.0,Normal,Long,13.0,3.0
1,1122072124,8186702651,519057,2024-08-01 00:03:00,2024-08-01 00:06:00,2024-08-01 00:18:00,\n\nAgent: Thank you for calling United Airlin...,calm,neutral,0.02,0.35,720.0,180.0,booking,0,0.0,Normal,Long,10.0,5.0
2,6834291559,2416856629,158319,2024-07-31 23:59:00,2024-08-01 00:07:00,2024-08-01 00:26:00,\n\nAgent: Thank you for calling United Airlin...,neutral,polite,-0.13,0.32,1140.0,480.0,irrops,23,0.0,Long,Long,1.0,1.0
3,2266439882,1154544516,488324,2024-08-01 00:05:00,2024-08-01 00:10:00,2024-08-01 00:17:00,\n\nAgent: Thank you for calling United Airlin...,neutral,frustrated,-0.20,0.20,420.0,300.0,upgrade,0,2.0,Normal,Normal,9.0,4.0
4,1211603231,5214456437,721730,2024-08-01 00:04:00,2024-08-01 00:14:00,2024-08-01 00:23:00,\n\nAgent: Thank you for calling United Airlin...,neutral,polite,-0.05,0.35,540.0,600.0,seating,0,0.0,Long,Normal,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66450,7569738090,7367304988,783441,2024-08-31 23:51:00,2024-08-31 23:57:00,2024-09-01 00:07:00,\n\nAgent: Thank you for calling United Airlin...,neutral,frustrated,-0.19,0.51,600.0,360.0,mileage plus,23,2.0,Normal,Normal,4.0,4.0
66451,1563273072,8022667294,413148,2024-08-31 23:48:00,2024-08-31 23:57:00,2024-09-01 00:25:00,\n\nAgent: Thank you for calling United Airlin...,calm,calm,0.05,0.34,1680.0,540.0,post flight,23,0.0,Long,Long,5.0,5.0
66452,8865997781,4474867021,980156,2024-08-31 23:55:00,2024-08-31 23:58:00,2024-09-01 00:06:00,\n\nAgent: Thank you for calling United Airlin...,calm,frustrated,0.03,0.22,480.0,180.0,upgrade,23,1.0,Normal,Normal,9.0,4.0
66453,8019240181,9762042472,616988,2024-08-31 23:52:00,2024-08-31 23:58:00,2024-09-01 00:04:00,\n\nAgent: Thank you for calling United Airlin...,calm,polite,0.05,0.42,360.0,360.0,upgrade,23,0.0,Normal,Normal,9.0,4.0


In [11]:
# Creating a new dataframe to analyse agents
filtered_df = merged_df[['agent_id', 'agent_tone', 'average_sentiment', 'AHT', 'primary_call_reason', 'category_by_reason']]
filtered_df

,agent_id,agent_tone,average_sentiment,AHT,primary_call_reason,category_by_reason
0,963118,neutral,-0.04,1860.0,voluntary cancel,3.0
1,519057,calm,0.02,720.0,booking,5.0
2,158319,neutral,-0.13,1140.0,irrops,1.0
3,488324,neutral,-0.20,420.0,upgrade,4.0
4,721730,neutral,-0.05,540.0,seating,3.0
...,...,...,...,...,...,...
66450,783441,neutral,-0.19,600.0,mileage plus,4.0
66451,413148,calm,0.05,1680.0,post flight,5.0
66452,980156,calm,0.03,480.0,upgrade,4.0
66453,616988,calm,0.05,360.0,upgrade,4.0


In [12]:
# Gathering unique agent IDs and sort them in ascending order
unique_agent_ids_sorted = pd.DataFrame(merged_df['agent_id'].unique(), columns=['agent_id']).sort_values(by='agent_id').reset_index(drop=True)
print(unique_agent_ids_sorted)


     agent_id
0      102574
1      103651
2      106000
3      107876
4      112609
..        ...
378    981776
379    981779
380    982131
381    992521
382    993862

[383 rows x 1 columns]


In [13]:
filtered_df = filtered_df.sort_values(by='agent_id').reset_index(drop=True)
filtered_df

,agent_id,agent_tone,average_sentiment,AHT,primary_call_reason,category_by_reason
0,102574,calm,0.01,6900.0,post flight,5.0
1,102574,neutral,-0.16,300.0,irrops,1.0
2,103651,calm,0.05,1260.0,voluntary change,2.0
3,103651,neutral,-0.02,300.0,baggage,3.0
4,103651,calm,0.04,300.0,check in,1.0
...,...,...,...,...,...,...
66450,993862,neutral,0.00,120.0,irrops,1.0
66451,993862,calm,0.11,120.0,baggage,3.0
66452,993862,neutral,-0.12,180.0,voluntary change,2.0
66453,993862,neutral,-0.14,1140.0,seating,3.0


In [14]:
# analysing agents so that we can rank them in future
agent_tone_counts = filtered_df.groupby('agent_id')['agent_tone'].value_counts().unstack(fill_value=0)
agent_tone_counts

agent_tone,angry,calm,frustrated,neutral,polite
agent_id,,,,,
102574,0,1,0,1,0
103651,2,98,12,171,0
106000,0,35,3,63,0
107876,0,1,0,5,0
112609,0,4,0,4,0
...,...,...,...,...,...
981776,1,98,20,160,1
981779,1,62,11,138,1
982131,2,78,12,159,0


In [15]:
# rating agents on the basis of their tone
# defining tone point system
# now for each value of tone assign points to agents to decide agent rating 
# polite: 5 
# calm: 4
# neutral: 3
# frustrated: 2 
# angry: 1

# after assigning the values multiply these factors to agent_tone_counts and divide by total number of calls handled by an agent 
# to decide agent rating 
tone_points = {
    'polite': 5,
    'calm': 4,
    'neutral': 3,
    'frustrated': 2,
    'angry': 1
}
agent_tone_counts_with_points = agent_tone_counts.copy()
for tone, points in tone_points.items():
    if tone in agent_tone_counts_with_points.columns:
        agent_tone_counts_with_points[tone] = agent_tone_counts_with_points[tone] * points
agent_total_points = agent_tone_counts_with_points.sum(axis=1)
agent_total_calls = agent_tone_counts.sum(axis=1)  # Original counts (without points)
agent_ratings = agent_total_points / agent_total_calls
agent_ratings

agent_id
102574    3.500000
103651    3.289753
106000    3.316832
107876    3.166667
112609    3.500000
            ...   
981776    3.278571
981779    3.239437
982131    3.247012
992521    3.232558
993862    3.280000
Length: 383, dtype: float64

In [16]:
# in above results we can see ratings of our agents
agent_ratings_df = agent_ratings.reset_index()
agent_ratings_df.columns = ['agent_id', 'agent_rating']
agent_ratings_df

,agent_id,agent_rating
0,102574,3.500000
1,103651,3.289753
2,106000,3.316832
3,107876,3.166667
4,112609,3.500000
...,...,...
378,981776,3.278571
379,981779,3.239437
380,982131,3.247012
381,992521,3.232558


In [17]:
# Sorting it 
unique_agents = agent_ratings_df.groupby('agent_id')['agent_rating'].mean().reset_index()

In [18]:
# Now ranking each agent according to their ratings
agent_ratings_sorted_df = unique_agents.sort_values(by='agent_rating', ascending=False)
agent_ratings_sorted_df['agent_rank'] = range(1, len(agent_ratings_sorted_df) + 1)
agent_ratings_sorted_df

,agent_id,agent_rating,agent_rank
120,391553,4.00,1
262,737543,4.00,2
53,239874,4.00,3
23,161354,4.00,4
314,846440,3.75,5
...,...,...,...
60,255256,3.00,379
184,547592,3.00,380
52,234327,3.00,381
91,336455,3.00,382


In [19]:
# we have sucessfully assigned ranks to the agents on the basis of their performance
# now lets catorize it in 5 different categories
num_agents = len(agent_ratings_sorted_df)
agents_per_category = num_agents // 5
agents_per_category

76

In [20]:
# we get 3 extras which we will put in category 1 
# assigning category
agent_ratings_sorted_df['category_of_agent'] = 0  # Initialize the category column
for i in range(num_agents):
    if i < agents_per_category * 4:  # For the first 4 categories
        agent_ratings_sorted_df.at[i, 'category_of_agent'] = (i // agents_per_category) + 1
    else:  # For the last category
        agent_ratings_sorted_df.at[i, 'category_of_agent'] = 5

agent_ratings_sorted_df['category_of_agent'] = agent_ratings_sorted_df['agent_rank'].apply(lambda x: min((x - 1) // agents_per_category + 1, 5))
agent_ratings_sorted_df

,agent_id,agent_rating,agent_rank,category_of_agent
120,391553,4.00,1,1
262,737543,4.00,2,1
53,239874,4.00,3,1
23,161354,4.00,4,1
314,846440,3.75,5,1
...,...,...,...,...
60,255256,3.00,379,5
184,547592,3.00,380,5
52,234327,3.00,381,5
91,336455,3.00,382,5


In [23]:
# we have successfully assigned categories to agents now we are going to merge this table to our merged_df main table on the basis of agent Id
final_df = pd.merge(merged_df, agent_ratings_sorted_df[['agent_id', 'category_of_agent']], on='agent_id', how='left')
final_df

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript,agent_tone,customer_tone,average_sentiment,...,AHT,AST,primary_call_reason,call_hour,elite_level_code,AST_Category,AHT_Category,rank,category_by_reason,category_of_agent
0,4667960400,2033123310,963118,2024-07-31 23:56:00,2024-08-01 00:03:00,2024-08-01 00:34:00,\n\nAgent: Thank you for calling United Airlin...,neutral,angry,-0.04,...,1860.0,420.0,voluntary cancel,23,4.0,Normal,Long,13.0,3.0,5
1,1122072124,8186702651,519057,2024-08-01 00:03:00,2024-08-01 00:06:00,2024-08-01 00:18:00,\n\nAgent: Thank you for calling United Airlin...,calm,neutral,0.02,...,720.0,180.0,booking,0,0.0,Normal,Long,10.0,5.0,5
2,6834291559,2416856629,158319,2024-07-31 23:59:00,2024-08-01 00:07:00,2024-08-01 00:26:00,\n\nAgent: Thank you for calling United Airlin...,neutral,polite,-0.13,...,1140.0,480.0,irrops,23,0.0,Long,Long,1.0,1.0,5
3,2266439882,1154544516,488324,2024-08-01 00:05:00,2024-08-01 00:10:00,2024-08-01 00:17:00,\n\nAgent: Thank you for calling United Airlin...,neutral,frustrated,-0.20,...,420.0,300.0,upgrade,0,2.0,Normal,Normal,9.0,4.0,1
4,1211603231,5214456437,721730,2024-08-01 00:04:00,2024-08-01 00:14:00,2024-08-01 00:23:00,\n\nAgent: Thank you for calling United Airlin...,neutral,polite,-0.05,...,540.0,600.0,seating,0,0.0,Long,Normal,3.0,3.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66450,7569738090,7367304988,783441,2024-08-31 23:51:00,2024-08-31 23:57:00,2024-09-01 00:07:00,\n\nAgent: Thank you for calling United Airlin...,neutral,frustrated,-0.19,...,600.0,360.0,mileage plus,23,2.0,Normal,Normal,4.0,4.0,5
66451,1563273072,8022667294,413148,2024-08-31 23:48:00,2024-08-31 23:57:00,2024-09-01 00:25:00,\n\nAgent: Thank you for calling United Airlin...,calm,calm,0.05,...,1680.0,540.0,post flight,23,0.0,Long,Long,5.0,5.0,1
66452,8865997781,4474867021,980156,2024-08-31 23:55:00,2024-08-31 23:58:00,2024-09-01 00:06:00,\n\nAgent: Thank you for calling United Airlin...,calm,frustrated,0.03,...,480.0,180.0,upgrade,23,1.0,Normal,Normal,9.0,4.0,1
66453,8019240181,9762042472,616988,2024-08-31 23:52:00,2024-08-31 23:58:00,2024-09-01 00:04:00,\n\nAgent: Thank you for calling United Airlin...,calm,polite,0.05,...,360.0,360.0,upgrade,23,0.0,Normal,Normal,9.0,4.0,1


In [25]:
print(final_df[['category_by_reason', 'category_of_agent']])

       category_by_reason  category_of_agent
0                     3.0                  5
1                     5.0                  5
2                     1.0                  5
3                     4.0                  1
4                     3.0                  4
...                   ...                ...
66450                 4.0                  5
66451                 5.0                  1
66452                 4.0                  1
66453                 4.0                  1
66454                 4.0                  5

[66455 rows x 2 columns]


In [28]:
# Here we can see that accurate agents are not assigned to calls
# like in 0 index row, we needed a 3rd category agent according to the reason of the call, but we assigned category 5 agent
# which means underuse of resources, on the other hand in index 3 
# we need a category 4 agent but we provided our category 1 agent 
# by using this we can assign categories to  the agents and make the work optimum

In [30]:
merged_df

,call_id,customer_id,agent_id,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript,agent_tone,customer_tone,average_sentiment,silence_percent_average,AHT,AST,primary_call_reason,call_hour,elite_level_code,AST_Category,AHT_Category,rank,category_by_reason
0,4667960400,2033123310,963118,2024-07-31 23:56:00,2024-08-01 00:03:00,2024-08-01 00:34:00,\n\nAgent: Thank you for calling United Airlin...,neutral,angry,-0.04,0.39,1860.0,420.0,voluntary cancel,23,4.0,Normal,Long,13.0,3.0
1,1122072124,8186702651,519057,2024-08-01 00:03:00,2024-08-01 00:06:00,2024-08-01 00:18:00,\n\nAgent: Thank you for calling United Airlin...,calm,neutral,0.02,0.35,720.0,180.0,booking,0,0.0,Normal,Long,10.0,5.0
2,6834291559,2416856629,158319,2024-07-31 23:59:00,2024-08-01 00:07:00,2024-08-01 00:26:00,\n\nAgent: Thank you for calling United Airlin...,neutral,polite,-0.13,0.32,1140.0,480.0,irrops,23,0.0,Long,Long,1.0,1.0
3,2266439882,1154544516,488324,2024-08-01 00:05:00,2024-08-01 00:10:00,2024-08-01 00:17:00,\n\nAgent: Thank you for calling United Airlin...,neutral,frustrated,-0.20,0.20,420.0,300.0,upgrade,0,2.0,Normal,Normal,9.0,4.0
4,1211603231,5214456437,721730,2024-08-01 00:04:00,2024-08-01 00:14:00,2024-08-01 00:23:00,\n\nAgent: Thank you for calling United Airlin...,neutral,polite,-0.05,0.35,540.0,600.0,seating,0,0.0,Long,Normal,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66450,7569738090,7367304988,783441,2024-08-31 23:51:00,2024-08-31 23:57:00,2024-09-01 00:07:00,\n\nAgent: Thank you for calling United Airlin...,neutral,frustrated,-0.19,0.51,600.0,360.0,mileage plus,23,2.0,Normal,Normal,4.0,4.0
66451,1563273072,8022667294,413148,2024-08-31 23:48:00,2024-08-31 23:57:00,2024-09-01 00:25:00,\n\nAgent: Thank you for calling United Airlin...,calm,calm,0.05,0.34,1680.0,540.0,post flight,23,0.0,Long,Long,5.0,5.0
66452,8865997781,4474867021,980156,2024-08-31 23:55:00,2024-08-31 23:58:00,2024-09-01 00:06:00,\n\nAgent: Thank you for calling United Airlin...,calm,frustrated,0.03,0.22,480.0,180.0,upgrade,23,1.0,Normal,Normal,9.0,4.0
66453,8019240181,9762042472,616988,2024-08-31 23:52:00,2024-08-31 23:58:00,2024-09-01 00:04:00,\n\nAgent: Thank you for calling United Airlin...,calm,polite,0.05,0.42,360.0,360.0,upgrade,23,0.0,Normal,Normal,9.0,4.0


In [38]:
# Reason Classifier using Call Transcript Classification & Clustering

In [40]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import nltk

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Load the dataset
df = pd.read_csv('output_file.csv')

# Clean the call transcript
stop_words = set(stopwords.words('english'))

def clean_transcript(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    tokens = word_tokenize(text)
    cleaned_text = ' '.join([word for word in tokens if word not in stop_words])
    return cleaned_text

df['cleaned_transcript'] = df['call_transcript'].apply(lambda x: clean_transcript(str(x)))

# Feature extraction using TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['cleaned_transcript']).toarray()

# Supervised Learning or Unsupervised Learning
if 'primary_call_reason' in df.columns:
    df['primary_call_reason'] = df['primary_call_reason'].str.lower().str.strip()
    y = pd.factorize(df['primary_call_reason'])[0]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Classification Accuracy: {accuracy * 100:.2f}%')
    
    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True)
    accuracy_data = pd.DataFrame(report).transpose()

    # Plotting F1 Score by Class
    plt.figure(figsize=(10, ))
    sns.barplot(x=accuracy_data.index[:-3], y=accuracy_data['f1-score'][:-3], palette='viridis')
    plt.title('F1 Score by Class')
    plt.xlabel('Class')
    plt.ylabel('F1 Score')
    plt.xticks(rotation=45)
    plt.show()

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues')
    plt.title('Confusion Matrix')
    plt.show()
    
else:
    kmeans = KMeans(n_clusters=5, random_state=42)
    kmeans.fit(X)
    df['cluster'] = kmeans.labels_
    
    # Plotting the cluster sizes
    cluster_counts = df['cluster'].value_counts()
    plt.figure(figsize=(10, 6))
    sns.barplot(x=cluster_counts.index, y=cluster_counts.values, palette='viridis')
    plt.title('Number of Samples in Each Cluster')
    plt.xlabel('Cluster')
    plt.ylabel('Number of Samples')
    plt.xticks(rotation=0)
    plt.show()

# Feature Importance
if 'primary_call_reason' in df.columns:
    feature_importances = clf.feature_importances_
    features = vectorizer.get_feature_names_out()
    importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
    importance_df = importance_df.sort_values(by='Importance', ascending=False).head(20)

    # Plotting the feature importances
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Importance', y='Feature', data=importance_df, palette='viridis')
    plt.title('Top 20 Important Features')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.show()

[nltk_data] Downloading package punkt to C:\Users\Auchitya
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Auchitya
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Classification Accuracy: 20.07%


C:\Users\Auchitya Sharma\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Auchitya Sharma\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Auchitya Sharma\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TypeError: Bbox.from_bounds() missing 1 required positional argument: 'height'